In [1]:
# Instalar a última versão do PySpark
!pip install pyspark #==3.3.1

# Instalar o NGROK
!wget -qnc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -n -q ngrok-stable-linux-amd64.zip


# Iniciar a sessão spark
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder                  
      .config('spark.ui.port', '4050')
      .appName("SparkSQL")
      .getOrCreate()
)

# Autenticar a sessão do SparkUI com NGROK
!./ngrok authtoken 2KBeQEmmd1YNlQ86GGKf3KFOkb3_6sQH7JEnvEhDxwn9A7WnT
get_ipython().system_raw('./ngrok http 4050 &')
!sleep 10
!curl -s http://localhost:4040/api/tunnels | grep -Po 'public_url":"(?=https)\K[^"]*'

ERROR: Invalid requirement: '#==3.3.1'

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip
'wget' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "C:\Users\dsant\anaconda3\lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "C:\Users\dsant\anaconda3\lib\site-packages\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "C:\Users\dsant\anaconda3\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from pyspark.sql.types import *

schema_remetente_destinatario = StructType([
    StructField('nome', StringType()),
    StructField('banco', StringType()),
    StructField('tipo', StringType())
])

schema_base_pix = StructType([
    StructField('id_transacao', IntegerType()),
    StructField('valor', DoubleType()),
    StructField('remetente', schema_remetente_destinatario),
    StructField('destinatario', schema_remetente_destinatario),
    StructField('chave_pix', StringType()),
    StructField('categoria', StringType()),
    StructField('transaction_date', StringType()),
    StructField('fraude', IntegerType())
])

caminho_json = 'drive/MyDrive/Colab Notebooks/case_final.json'

df = spark.read.json(
    caminho_json,
    schema=schema_base_pix,
    timestampFormat="yyyy-MM-dd HH:mm:ss"
)

In [ ]:
df.printSchema()

df.show()

In [ ]:
from pyspark.sql.functions import *

df_flatten = df.withColumns({
    'destinatario_nome': col('destinatario').getField('nome'),
    'destinatario_banco': col('destinatario').getField('banco'),
    'destinatario_tipo': col('destinatario').getField('tipo'),
}).drop('remetente', 'destinatario')

In [ ]:
df_flatten.printSchema()

In [ ]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(
    inputCols=[
        'destinatario_nome',
        'destinatario_banco',
        'destinatario_tipo',
        'categoria',
        'chave_pix'
    ],
    outputCols=[
        'destinatario_nome_index',
        'destinatario_banco_index',
        'destinatario_tipo_index',
        'categoria_index',
        'chave_pix_index'        
    ]
)

df_index = indexer.fit(df_flatten).transform(df_flatten)

In [ ]:
df_index.printSchema()

In [ ]:
df_index.show()

In [ ]:
is_fraud = df_index.filter("fraude == 1") # filter(col('fraude') == 1)
no_fraud = df_index.filter("fraude == 0")

In [ ]:
no_fraud = no_fraud.sample(False, 0.01, seed=123) 

In [ ]:
df_concat = no_fraud.union(is_fraud)
df = df_concat.sort("transaction_date")
df.count()

In [ ]:
train, test = df.randomSplit([0.7, 0.3], seed = 123)
print("train =", train.count(), " test =", test.count())

In [ ]:
is_fraud = udf(lambda fraud: 1.0 if fraud > 0 else 0.0, DoubleType())
train = train.withColumn("is_fraud", is_fraud(train.fraude))

In [ ]:
train.printSchema()

In [ ]:
train.columns

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

assembler = VectorAssembler(
    inputCols = [x for x in train.columns if x not in ['transaction_date', 'fraude', 'is_fraud', 'destinatario_nome', 'destinatario_banco', 'destinatario_tipo', 'chave_pix', 'categoria']],
    outputCol="features"
)

In [ ]:
lr = LogisticRegression().setParams(
    maxIter=100000,
    labelCol = "is_fraud",
    predictionCol="prediction"
)

In [ ]:
model = Pipeline(stages=[assembler, lr]).fit(train)

In [ ]:
predicted = model.transform(test)

In [ ]:
predicted.show()

In [ ]:
predicted = predicted.withColumn('is_fraud', is_fraud(predicted.fraude))
predicted.crosstab('is_fraud', 'prediction').show()

In [ ]:
df_teste_cols = [
    'id_transacao',
    'valor', 
    'transaction_date',
    'destinatario_nome_index',
    'destinatario_banco_index',
    'destinatario_tipo_index',
    'chave_pix_index',
    'categoria_index',
    'fraude'
]

df_teste_data = [
    (999,103.2, "2023-01-01 11:56:41", 328.0, 4.0, 1.0, 3.0, 5.0, 0),
    (998, 500000.0, "2023-01-01 11:56:41", 328.0, 2.0, 3.0, 2.0, 5.0, 1),
    (997, 19999.0, "2023-01-01 11:56:41", 328.0, 1.0, 2.0, 1.0, 5.0, 0),
]

df_teste = spark.createDataFrame(df_teste_data).toDF(*df_teste_cols)

In [ ]:
new_prediction = model.transform(df_teste)

In [ ]:
new_prediction.show()